In [ ]:
%load_ext autoreload

%autoreload 2
from datasets import load_dataset, concatenate_datasets, DatasetDict, ClassLabel
ag_news = load_dataset("ag_news")
startup_news = load_dataset("csv", data_files="vcnewsdaily_clean.csv", split="train")

In [108]:
features = ag_news["train"].features.copy()
features["label"] = ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech', "Startup"])
def adjust_labels(batch):
    return batch
ag_dataset = ag_news.map(adjust_labels, batched=True, features=features)
startup_dataset = startup_news.map(adjust_labels, batched=True, features=features)

combined_train_dataset = concatenate_datasets([ag_dataset['train'], startup_dataset['train']])
combined_test_dataset = concatenate_datasets([ag_dataset['test'], startup_dataset['test']])
news_dataset = DatasetDict({
    'train': combined_train_dataset,
    'test': combined_test_dataset
})


Loading cached processed dataset at /Users/ivan/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-04a7b99e10819c7f.arrow
Loading cached processed dataset at /Users/ivan/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-cd075d1462560cf4.arrow
Loading cached processed dataset at /Users/ivan/.cache/huggingface/datasets/csv/default-a9cb4e7ee47f8307/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-fc85598892306f35.arrow
Loading cached processed dataset at /Users/ivan/.cache/huggingface/datasets/csv/default-a9cb4e7ee47f8307/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-9ed70490a7791cfc.arrow


In [109]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
news_dataset = news_dataset.map(preprocess_function, batched=True)

In [110]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [111]:
id2label = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech", 4: "Startup"}
label2id = {"World": 0, "Sports": 1, "Business": 2, "Sci/Tech": 3, "Startup": 4}

In [112]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.

In [114]:
from datasets import load_dataset, concatenate_datasets

training_args = TrainingArguments(
    output_dir="news_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    use_mps_device=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=news_dataset["train"],
    eval_dataset=news_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/ivan/Desktop/vc-news-hunter/classification/news_classifier is already a clone of https://huggingface.co/ivanzhang/news_classifier. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/ivan/Desktop/vc-news-hunter/classification/venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  0%|          | 0/16904 [02:48<?, ?it/s]          

{'loss': 0.4329, 'learning_rate': 1.940842404164695e-05, 'epoch': 0.06}


...


  0%|          | 0/16904 [07:54<?, ?it/s]           

{'loss': 0.2204, 'learning_rate': 1.7633696166587792e-05, 'epoch': 0.24}



  0%|          | 0/16904 [09:32<?, ?it/s]           

{'loss': 0.2066, 'learning_rate': 1.7042120208234738e-05, 'epoch': 0.3}


KeyboardInterrupt: 